In [15]:
from rap_scrpr import *
from rap_db import *

from os import listdir
from os.path import isfile, join
import nltk
#nltk.download()
estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

create_music_tables(estconn, bypass = True)
bulk_load(estconn, ['mf_doom', 'chance_the_rapper'])

In [5]:
doom = basic_lyrc_pull(estconn, 'MF Doom')

In [6]:
#
#estconn.close()

In [7]:
doom.keys()

dict_keys(['Absolutely', 'Appetizers: One Beer', 'Ballskin', 'Batty-Boys', 'Beef Rapp', 'Blunt Drunk (All Outta Ale Remix)', 'Dead Bent', 'Deep Fried Frenz', 'Doomsday', 'Entrees: Kon Karne', 'Entrees: Kookies', 'Entrees: Vomitspit', 'Gas Drawls', 'Go With the Flow', 'Greenbacks', 'Hey!', 'Hoe Cakes', 'Impostas', 'Is He Ill?', 'Kon Queso', 'Lightworks', 'Meat Grinder', 'Microwave Mayo', "More Rhymin'", 'Rap Ambush', 'Red &amp; Gold', "That's That", 'The Finest'])

In [8]:
doom['Beef Rapp']

"\n* send corrections to the typist\n\n[MF Doom]\nBeef rap\nCould lead to gettin teeth capped\nOr even a wreath for mom dukes on some grief crap\nI suggest ya change ya diet\nIt can lead ta high blood pressure if ya fry it\nOr even a stroke, heart attack, heart disease\nIt ain't no startin back once arteries start ta squeeze\nTake the easy way out phony, until then\nThey know they wouldn't be talkin that bologna in the bullpen\nSo disgustin, pardon self as I discuss this\nThey talk a wealth of shit and they ain't never seen the justice\nBust this, like a cold milk from out the toilet\nTwo batteries some Brillo and some foil, he'a boil it\nHe be better off on PC glued\nAnd it's a feud so don't be in no TV mood\nEvery week it's mystery meat, seaweed stewed [food, we need food!]\n\nHe wears a mask just to cover the raw flesh\nA rather ugly brother with flows that's gorgeous\nDrop dead joints hit the whips like bird shit\nThey need it like a hole in they head or a third tit\nHer bra smell,

In [19]:
#can't just blindly tokenize, you end up hurting info
for k, v in doom.items():
    print(v)


"Things take a turn for the worse"
  "Send him back where he came from"
"We got shots fired down the street"

[Doom]
Ba-di-yahhhh...
"Alright, steady"
Ba-di-yah, ba-di-yah

Absolute power corrupts absolutely
If they get the Villain surrounded {*AHEM*}

Good to the last drop nigga
Hood 'til the last pop them bigot-ass piglets
Go for the bosses, dicks or higher
Beatwalker still get caught up in the crossfire
Get at the sarge, he give the orders
The rest of them punks'll quit the force or get slaughtered
Use stealth, play alleys, jet
We'd rather not alert them gnomes and federales, yet
Keep the local random cases in each state
Escape to the beaches for the season and be straight
You on call, keep your Nikes on tight, no white
Anybody tell get lighted on sight
The ringer's on a need-to-know basis
No persons, places, things and no faces
Get bagged, you're on your own, act it alone
Back home your fam'll be provided for while you're gone
It's a price on all snitches
If you nice bring back ey

#can't just blindly tokenize, you end up hurting info
for k, v in doom.items():
    tokens = nltk.word_tokenize(v)
    print(k, v, '\n', tokens)
    break

In [ ]:
#need to find the best way to mark of important parts of the song,
# {**} == not sure but one thing is can mean someone saying something
# {?} == dont know line
# [Artist] == i think this is when an artist s speaking
# [yes] == can also mean like () whe someone says somthing, have to use [Capital First letter] to differentiate
#or a great way to differentiate is to check contents of [] against db of artists with tolower and edge case for chorus or outro or intro
# * == annotator notes
# (action!) == i think this is sound of action, can also be someone saying something in track
# () == someone saying something or repeat signal
# (**) ==
# looks like () and [] and {} are pretty much interchanchable and it matters a lot more whats inside
# [???] == i think it means they didn't know
# "" == i think this is when someone else is talking over the track
# Man1: denoting that someone is speaking in a conversation
words = re.sub("([\(\[]).*?([\)\]])", "", lyrics)
words = re.sub("\n","--",words)


In [ ]:
estconn.close()